## Exploring the LEDGAR dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [1]:
import os

# provide your own OpenAI API key here
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxx"

#### Install the autolabel library

In [2]:
!pip install 'refuel-autolabel[openai]'

#### Download the dataset

In [3]:
from autolabel import get_data

get_data("ledgar")

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100% [........................................................] 139187 / 139187

100% [......................................................] 1376364 / 1376364

This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

### First labeling run

In [4]:
import json

from autolabel import LabelingAgent

In [5]:
# load the config
with open("config_ledgar.json") as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `classification` (since it's a classification task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: `'You are an expert at understanding legal contracts...` (how we describe the task to the LLM)
* `prompt.labels`: `['Agreements',
   'Amendments',
   'Anti-Corruption Laws',
   'Applicable Laws',
   'Approvals',
   'Arbitration',
   'Assignments',
   'Assigns', ...]` (the full list of labels to choose from)
* `prompt.few_shot_num`: 4 (how many labeled examples to provide to the LLM)

In [8]:
config

{'task_name': 'LegalProvisionsClassification',
 'task_type': 'classification',
 'dataset': {'label_column': 'label', 'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are an expert at understanding legal contracts. Your job is to correctly classify legal provisions in contracts into one of the following categories.\nCategories:{labels}\n',
  'labels': ['Agreements',
   'Amendments',
   'Anti-Corruption Laws',
   'Applicable Laws',
   'Approvals',
   'Arbitration',
   'Assignments',
   'Assigns',
   'Authority',
   'Authorizations',
   'Base Salary',
   'Benefits',
   'Binding Effects',
   'Books',
   'Brokers',
   'Capitalization',
   'Change In Control',
   'Closings',
   'Compliance With Laws',
   'Confidentiality',
   'Consent To Jurisdiction',
   'Consents',
   'Construction',
   'Cooperation',
   'Costs',
   'Counterparts',
   'Death',
   'Defined Terms',
   'Definitions',
   'Disability',
   'Disclosures',
   'Duties

In [10]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [11]:
from autolabel import AutolabelDataset

ds = AutolabelDataset("test.csv", config=config)
agent.plan(ds)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $8.3316 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0042 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at understanding legal contracts. Your job is to correctly classify legal provisions in contracts into one of the following categories.
Categories:Agreements
Amendments
Anti-Corruption Laws
Applicable Laws
Approvals
Arbitration
Assignments
Assigns
Authority
Authorizations
Base Salary
Benefits
Binding Effects
Books
Brokers
Capitalization
Change In Control
Closings
Compliance With Laws
Confidentiality
Consent To Jurisdiction
Consents
Construction
Cooperation
Costs
Counterparts
Death
Defined Terms
Definitions
Disability
Disclosures
Duties
Effective Dates
Effectiveness
Employment
Enforceability
Enforcements
Entire Agreements
Erisa
Existence
Expenses
Fees
Financial Statements
Forfeitures
Further Assurances
General
Governing Laws
Headings
Indemnifications
Indemnity
Insurances
Integration
Intellectual Property
Interests
Interpretations
Jurisdictions
Liens
Litigations
Miscellaneous
Modifications
No Conflicts
No Defaults
No Waivers
Non-Disparagement
Notices
Organizations
Parti

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [12]:
# now, do the actual labeling
ds = agent.run(ds, max_items=100)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

2023-06-14 13:31:32 openai INFO: error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 9ff1d233ffc8b6842e6039eeb99aa39e in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-06-14 13:31:32 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 9ff1d233ffc8b6842e6039eeb99aa39e in your message.).


/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

2023-06-14 13:31:55 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88790 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-06-14 13:31:55 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88790 / min. Contact us through our help center at help.openai.com if you continue to have issues..
2023-06-14 13:31:57 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens p

2023-06-14 13:32:17 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89085 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-06-14 13:32:17 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89085 / min. Contact us through our help center at help.openai.com if you continue to have issues..
2023-06-14 13:32:19 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens p

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Actual Cost: 0.2181


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 100     │ -inf      │ 0.71     │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

We are at 71% accuracy when labeling the first 100 examples. Let's see if we can use confidence scores to improve accuracy further by removing the less confident examples from our labeled set.

### Compute confidence scores


In [24]:
# Start computing confidence scores (using Refuel's LLMs)
os.environ["REFUEL_API_KEY"] = "xxxxxxxxxxxxxxxxx"

In [25]:
# set `compute_confidence` -> True
config["model"]["compute_confidence"] = True

In [26]:
agent = LabelingAgent(config=config)

In [27]:
from autolabel import AutolabelDataset

ds = AutolabelDataset("data/ledgar/test.csv", config=config)
agent.plan(ds)

2023-06-14 15:03:28 langchain.embeddings.openai WARNING: Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $8.334  │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0042 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at understanding legal contracts. Your job is to correctly classify legal provisions in contracts into one of the following categories.
Categories:Agreements
Amendments
Anti-Corruption Laws
Applicable Laws
Approvals
Arbitration
Assignments
Assigns
Authority
Authorizations
Base Salary
Benefits
Binding Effects
Books
Brokers
Capitalization
Change In Control
Closings
Compliance With Laws
Confidentiality
Consent To Jurisdiction
Consents
Construction
Cooperation
Costs
Counterparts
Death
Defined Terms
Definitions
Disability
Disclosures
Duties
Effective Dates
Effectiveness
Employment
Enforceability
Enforcements
Entire Agreements
Erisa
Existence
Expenses
Fees
Financial Statements
Forfeitures
Further Assurances
General
Governing Laws
Headings
Indemnifications
Indemnity
Insurances
Integration
Intellectual Property
Interests
Interpretations
Jurisdictions
Liens
Litigations
Miscellaneous
Modifications
No Conflicts
No Defaults
No Waivers
Non-Disparagement
Notices
Organizations
Parti

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [28]:
ds = agent.run(ds, max_items=100)

2023-06-14 15:04:00 autolabel.labeler INFO: Task run already exists.


There is an existing task with following details: id='3001625978' created_at=datetime.datetime(2023, 6, 14, 14, 42,
51, 302325) task_id='0890358f0f66b2f3e1284f6a180dc84a' dataset_id='a7f684e9cdafe3181a12b46035965fbe' 
current_index=100 output_file='test_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None

Evaluating the existing task...

Metric: auroc: 0.5


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 100     │ -inf      │ 0.71     │ 1.0             │
│ 100     │ 0.5       │ 0.71     │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

...

2023-06-14 15:08:29 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that!' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-06-14 15:08:29 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that!",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that!', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Wed, 14 Jun 2023 22:08:29 GMT', 'Content-Type': 'application/json', 'Content-Length': '176', 'Connection': 'keep-alive', 'access-control-allow-origin':

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Metric: auroc: 0.7217
Actual Cost: 0.01


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 100     │ -inf      │ 0.71     │ 1.0             │
│ 1       │ 0.9996    │ 1.0      │ 0.01            │
│ 14      │ 0.9969    │ 1.0      │ 0.14            │
│ 15      │ 0.9965    │ 0.9333   │ 0.15            │
│ 16      │ 0.9964    │ 0.9375   │ 0.16            │
│ 18      │ 0.9953    │ 0.8333   │ 0.18            │
│ 31      │ 0.9868    │ 0.9032   │ 0.31            │
│ 32      │ 0.9866    │ 0.875    │ 0.32            │
│ 35      │ 0.9759    │ 0.8857   │ 0.35            │
│ 36      │ 0.9759    │ 0.8611   │ 0.36            │
│ 39      │ 0.9725    │ 0.8718   │ 0.39            │
│ 40      │ 0.972     │ 0.85     │ 0.4             │
│ 43      │ 0.9635    │ 0.8605   │ 0.43            │
│ 44      │ 0.9558    │ 0.8409   │ 0.44            │
│ 48      │ 0.9539    │ 0.8542   │ 0.48            │
│ 49      │ 0.9476    │ 0.8367   │ 0.49            │
│ 51      │ 0.9422    │ 0.8431   │ 0.51            │
│ 52      │ 0.9342    │ 0.8269   │ 0.52            │
│ 56      │ 0.9226    │ 0.8393   │ 0.56            │
│ 57      │ 0.9201    │ 0.8246   │ 0.57            │
│ 58      │ 0.9167    │ 0.8276   │ 0.58            │
│ 59      │ 0.9158    │ 0.8136   │ 0.59            │
│ 62      │ 0.8988    │ 0.8226   │ 0.62            │
│ 63      │ 0.8985    │ 0.8095   │ 0.63            │
│ 68      │ 0.8268    │ 0.8235   │ 0.68            │
│ 70      │ 0.8003    │ 0.8      │ 0.7             │
│ 73      │ 0.755     │ 0.8082   │ 0.73            │
│ 76      │ 0.7465    │ 0.7763   │ 0.76            │
│ 77      │ 0.7354    │ 0.7792   │ 0.77            │
│ 78      │ 0.734     │ 0.7692   │ 0.78            │
│ 82      │ 0.7119    │ 0.7805   │ 0.82            │
│ 86      │ 0.6721    │ 0.7442   │ 0.86            │
│ 88      │ 0.5911    │ 0.75     │ 0.88            │
│ 89      │ 0.5795    │ 0.7416   │ 0.89            │
│ 90      │ 0.5614    │ 0.7444   │ 0.9             │
│ 92      │ 0.53      │ 0.7283   │ 0.92            │
│ 94      │ 0.5223    │ 0.734    │ 0.94            │
│ 97      │ 0.3197    │ 0.7113   │ 0.97            │
│ 98      │ 0.1733    │ 0.7143   │ 0.98            │
│ 99      │ 0.0402    │ 0.7071   │ 0.99            │
│ 100     │ 0.02      │ 0.71     │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

Looking at the table above, we can see that if we set the confidence threshold at `0.755`, we are able to label at 80.82% accuracy and getting a completion rate of 73%. This means, we would ignore all the data points where confidence score is less than `0.755` (which would end up being around 27% of all samples). This would, however, guarantee a very high quality labeled dataset for us. 